In [1]:
import pandas as pd
import os
import requests as re
import datetime
import time
from IPython.display import JSON
import json

# Import Dataframes

### Flights

In [2]:
pd.set_option('max_columns', None)
#pd.reset_option('max_columns')

In [3]:
# Import df_flights. Specify datatype of cancellation_code so there are not mixed datatypes present.
df_flights = pd.read_csv('flights_data.csv', dtype={'cancellation_code':'object'})

In [4]:
# Check flight delay bins
pd.cut(df_flights['dep_delay'], bins=10).value_counts()

(-50.801, 131.1]    312103
(131.1, 311.2]        6333
(311.2, 491.3]         753
(491.3, 671.4]         174
(671.4, 851.5]         129
(851.5, 1031.6]         88
(1031.6, 1211.7]        35
(1391.8, 1571.9]        20
(1211.7, 1391.8]        16
(1571.9, 1752.0]         6
Name: dep_delay, dtype: int64

In [5]:
# Filter out extreme delays which account for less than 1% of flights, for more accurate means
df_flights = df_flights[df_flights['dep_delay'] < 311.2]

In [6]:
df_flights.head(1)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-01-01,DL,DL_CODESHARE,DL,3468,9E,N292PQ,3468,15380,TVC,"Traverse City, MI",11433,DTW,"Detroit, MI",615,611.0,-4.0,26.0,637.0,712.0,9.0,738,721.0,-17.0,0.0,NaN,0.0,N,83.0,70.0,35.0,1.0,207.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Flights_test

In [7]:
# Look through flight data from January 2020
df_flights_test = pd.read_csv('flights_test.csv')

# Move column names to first row of data
df_flights_test = (df_flights_test.T.reset_index().T.reset_index(drop=True))

# Define new column names
df_flights_test.columns = ['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance']

# Clean up fl_date
df_flights_test['fl_date'] = df_flights_test['fl_date'].str[0:10]

In [8]:
df_flights_test.head(1)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,2020-01-01,WN,WN.1,WN.2,5888,WN.3,N951WN,5888.1,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363


# Weather Lookup

Usage of weather data to estimate flight delay will depend on if there is an API with sufficient call allowances. Currently pending approval for student account on [OpenWeather](https://openweathermap.org/api).

In [9]:
# Get hour of departure and arrival times (approximate using crs for 2020)
df_flights['hour_of_day_dep'] = df_flights['dep_time'] // 100
df_flights['hour_of_day_arr'] = df_flights['arr_time'] // 100
df_flights_test['hour_of_day_dep'] = df_flights_test['crs_dep_time'].astype(int) // 100
df_flights_test['hour_of_day_arr'] = df_flights_test['crs_arr_time'].astype(int) // 100

# Set delay nulls to zero (zero minutes of delay instead of null)
df_flights[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']] = df_flights[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].fillna(value=0)

In [10]:
# Create dataframes for one-day API lookups
df_flights_20190101 = df_flights[df_flights['fl_date'] == '2019-01-01']
df_flights_20200101 = df_flights_test[df_flights_test['fl_date'] == '2020-01-01']

In [11]:
df_flights_20190101.head(1)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,hour_of_day_dep,hour_of_day_arr
154049,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,"Myrtle Beach, SC",11057,CLT,"Charlotte, NC",2157,2158.0,1.0,10.0,2208.0,2248.0,20.0,2311,2308.0,-3.0,0.0,NaN,0.0,N,74.0,70.0,40.0,1.0,157.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,21.0,23.0


In [12]:
df_flights_20200101.head(1)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,hour_of_day_dep,hour_of_day_arr
0,2020-01-01,WN,WN.1,WN.2,5888,WN.3,N951WN,5888.1,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363,18,19


In [13]:
# Set up API query function
api_key = os.environ["VisualCrossing_API_Code"]

def weather_query_oneday(flightsDF):
    '''
    Given a dataframe of flights info on a given day, return a list of weather conditions for each unique origin city.
    '''
    
    city_list = flightsDF['origin_city_name'].unique().tolist()
    city_list_cleaned = []
    
    # Replace forward slashes in city names
    for elem in city_list:
        if elem.find('/') == -1:
            city_list_cleaned.append(elem)
        else:
            city_list_cleaned.append(elem.replace('/','-'))
    
    date = flightsDF['fl_date'].iloc[0]
    
    weather_conditions_list = []
    weather_conditions_dict = {}

    for i in range(0,len(city_list_cleaned)):
        res = re.get(f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city_list_cleaned[i]}/{date}?unitGroup=us&key={api_key}&include=current')
        weather_conditions_list.append(res.json()['days'][0]['conditions'])
        weather_conditions_dict.update({city_list[i]:weather_conditions_list[i]})
    
    return weather_conditions_dict

In [14]:
def weather_simplification_dict(weather_dict):
    '''
    Given a list of weather conditions in a dictionary with cities, simplify the types into Sunny, Cloudy, Rainy, or Snow.
    '''

    weather_simplified = weather_dict.copy()

    for key, value in weather_dict.items():
        
        if value.find('Snow') > -1:
            weather_simplified.update({key:'Snow'})
            
        elif value.find('Rain') > -1:
            weather_simplified.update({key:'Rainy'})
            
        elif (value.find('cloudy') > -1) or (value.find('Overcast') > -1):
            weather_simplified.update({key:'Cloudy'})
            
        else:
            weather_simplified.update({key:'Sunny'})
    
    return weather_simplified

In [15]:
# # Perform API queries (I did this and exported the results)
# df_flights_20190101_weather = weather_query_oneday(df_flights_20190101)
# df_flights_20200101_weather = weather_query_oneday(df_flights_20200101)

In [16]:
df_flights_20190101_weather = json.load(open('df_flights_20190101_weather.json', 'r'))
df_flights_20200101_weather = json.load(open('df_flights_20200101_weather.json', 'r'))

In [17]:
# Simplify the weather results
df_flights_20190101_weather_simplified = weather_simplification_dict(df_flights_20190101_weather)
df_flights_20200101_weather_simplified = weather_simplification_dict(df_flights_20200101_weather)

In [18]:
# Apply weather results to df_flights_20190101
df_flights_20190101['origin_weather'] = df_flights_20190101.apply(lambda row: df_flights_20190101_weather_simplified[row['origin_city_name']], axis=1)


F:\Anaconda\envs\pycaret\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
# Find mean weather day for the 4 possible conditions
weather_delay_means = df_flights_20190101.groupby('origin_weather').mean()['weather_delay']
weather_delay_std = df_flights_20190101.groupby('origin_weather').std()['weather_delay']
weather_delay_means

origin_weather
Cloudy    0.304863
Rainy     0.170538
Snow      3.141741
Sunny     0.099901
Name: weather_delay, dtype: float64

In [20]:
# Make three assumptions due to API limitations:
# 1.  2018/2019 weather for the first 8 days is the same as 2019-01-01
# 2.  2020      weather for the first 8 days is the same as 2020-01-01
# 3.  If a city's weather is not available, assume it is Sunny.


# Apply the weather to the remaining days of the dataframe, apply the mean delays, and set any missing cities' delay equal to 0.

df_flights['origin_weather'] = df_flights.apply(lambda row: df_flights_20190101_weather_simplified[row['origin_city_name']] if row['origin_city_name'] in df_flights_20190101_weather_simplified.keys() else 'Sunny', axis=1)
df_flights['mean_weather_delay'] = df_flights.apply(lambda row: weather_delay_means[row['origin_weather']], axis=1)
df_flights['std_weather_delay'] = df_flights.apply(lambda row: weather_delay_std[row['origin_weather']], axis=1)

df_flights_test['origin_weather'] = df_flights_test.apply(lambda row: df_flights_20200101_weather_simplified[row['origin_city_name']] if row['origin_city_name'] in df_flights_20200101_weather_simplified.keys() else 'Sunny', axis=1)
df_flights_test['mean_weather_delay'] = df_flights_test.apply(lambda row: weather_delay_means[row['origin_weather']], axis=1)
df_flights_test['std_weather_delay'] = df_flights_test.apply(lambda row: weather_delay_std[row['origin_weather']], axis=1)

In [21]:
df_flights;

# Feature Engineering

In [22]:
# Exclude diverted and cancelled flights
df_flights = df_flights[df_flights['diverted'] != 1]
df_flights = df_flights[df_flights['cancelled'] != 1]

# Format fl_date column as a date
df_flights['fl_date'] = pd.to_datetime(df_flights['fl_date'])
df_flights_test['fl_date'] = pd.to_datetime(df_flights_test['fl_date'])

# Create 'state' column
df_flights['state'] = df_flights['origin_city_name'].str[-2:]
df_flights_test['state'] = df_flights_test['origin_city_name'].str[-2:]

# Create fl_day column
df_flights['fl_day'] = pd.DatetimeIndex(df_flights['fl_date']).day
df_flights_test['fl_day'] = pd.DatetimeIndex(df_flights_test['fl_date']).day

# Get and apply daily mean flight delay values
df_daily_delay_means = df_flights.groupby('fl_day').mean()[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].reset_index()
df_flights['daily_arr_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means['arr_delay'][row['fl_day']-1], axis=1)
df_flights['daily_carrier_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means['carrier_delay'][row['fl_day']-1], axis=1)
df_flights['daily_weather_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means['weather_delay'][row['fl_day']-1], axis=1)
df_flights['daily_nas_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means['nas_delay'][row['fl_day']-1], axis=1)
df_flights['daily_security_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means['security_delay'][row['fl_day']-1], axis=1)
df_flights['daily_late_aircraft_delay_mean'] = df_flights.apply(lambda row: df_daily_delay_means['late_aircraft_delay'][row['fl_day']-1], axis=1)

df_flights_test['daily_arr_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means['arr_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_carrier_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means['carrier_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_weather_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means['weather_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_nas_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means['nas_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_security_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means['security_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_late_aircraft_delay_mean'] = df_flights_test.apply(lambda row: df_daily_delay_means['late_aircraft_delay'][row['fl_day']-1], axis=1)

# Get and apply daily standard deviation of flight delay values
df_daily_delay_std = df_flights.groupby('fl_day').std()[['arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay']].reset_index()
df_flights['daily_arr_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std['arr_delay'][row['fl_day']-1], axis=1)
df_flights['daily_carrier_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std['carrier_delay'][row['fl_day']-1], axis=1)
df_flights['daily_weather_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std['weather_delay'][row['fl_day']-1], axis=1)
df_flights['daily_nas_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std['nas_delay'][row['fl_day']-1], axis=1)
df_flights['daily_security_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std['security_delay'][row['fl_day']-1], axis=1)
df_flights['daily_late_aircraft_delay_std'] = df_flights.apply(lambda row: df_daily_delay_std['late_aircraft_delay'][row['fl_day']-1], axis=1)

df_flights_test['daily_arr_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std['arr_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_carrier_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std['carrier_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_weather_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std['weather_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_nas_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std['nas_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_security_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std['security_delay'][row['fl_day']-1], axis=1)
df_flights_test['daily_late_aircraft_delay_std'] = df_flights_test.apply(lambda row: df_daily_delay_std['late_aircraft_delay'][row['fl_day']-1], axis=1)

In [23]:
# Get and apply hourly mean flight delay values
dep_hourly_delay_mean = df_flights.groupby('hour_of_day_dep').mean()['dep_delay']
arr_hourly_delay_mean = df_flights.groupby('hour_of_day_arr').mean()['arr_delay']

df_flights['dep_mean_hourly_delay'] = df_flights.apply(lambda row: dep_hourly_delay_mean[row['hour_of_day_dep']], axis=1)
df_flights['arr_mean_hourly_delay'] = df_flights.apply(lambda row: arr_hourly_delay_mean[row['hour_of_day_arr']], axis=1)
df_flights_test['dep_mean_hourly_delay'] = df_flights_test.apply(lambda row: dep_hourly_delay_mean[row['hour_of_day_dep']], axis=1)
df_flights_test['arr_mean_hourly_delay'] = df_flights_test.apply(lambda row: arr_hourly_delay_mean[row['hour_of_day_arr']], axis=1)

# Get and apply hourly standard deviation of flight delay values
dep_hourly_delay_std = df_flights.groupby('hour_of_day_dep').std()['dep_delay']
arr_hourly_delay_std = df_flights.groupby('hour_of_day_arr').std()['arr_delay']

df_flights['dep_std_hourly_delay'] = df_flights.apply(lambda row: dep_hourly_delay_std[row['hour_of_day_dep']], axis=1)
df_flights['arr_std_hourly_delay'] = df_flights.apply(lambda row: arr_hourly_delay_std[row['hour_of_day_arr']], axis=1)
df_flights_test['dep_std_hourly_delay'] = df_flights_test.apply(lambda row: dep_hourly_delay_std[row['hour_of_day_dep']], axis=1)
df_flights_test['arr_std_hourly_delay'] = df_flights_test.apply(lambda row: arr_hourly_delay_std[row['hour_of_day_arr']], axis=1)

In [24]:
# Calculate and apply mean marketing (mkt) and operating (op) carrier delays
df_mean_mkt_carrier_delays = df_flights.groupby('mkt_unique_carrier').mean()['carrier_delay']
df_mean_op_carrier_delays = df_flights.groupby('op_unique_carrier').mean()['carrier_delay']
df_flights['mean_mkt_carrier_delay'] = df_flights.apply(lambda row: df_mean_mkt_carrier_delays[row['mkt_unique_carrier']], axis=1)
df_flights['mean_op_carrier_delay'] = df_flights.apply(lambda row: df_mean_op_carrier_delays[row['op_unique_carrier']], axis=1)
df_flights_test['mean_mkt_carrier_delay'] = df_flights_test.apply(lambda row: df_mean_mkt_carrier_delays[row['mkt_unique_carrier']], axis=1)
df_flights_test['mean_op_carrier_delay'] = df_flights_test.apply(lambda row: df_mean_op_carrier_delays[row['op_unique_carrier']] if row['op_unique_carrier'] in df_mean_op_carrier_delays.index else df_mean_op_carrier_delays.mean(), axis=1)

# Calculate and apply standard deviation of marketing (mkt) and operating (op) carrier delays
df_std_mkt_carrier_delays = df_flights.groupby('mkt_unique_carrier').std()['carrier_delay']
df_std_op_carrier_delays = df_flights.groupby('op_unique_carrier').std()['carrier_delay']
df_flights['std_mkt_carrier_delay'] = df_flights.apply(lambda row: df_std_mkt_carrier_delays[row['mkt_unique_carrier']], axis=1)
df_flights['std_op_carrier_delay'] = df_flights.apply(lambda row: df_std_op_carrier_delays[row['op_unique_carrier']], axis=1)
df_flights_test['std_mkt_carrier_delay'] = df_flights_test.apply(lambda row: df_std_mkt_carrier_delays[row['mkt_unique_carrier']], axis=1)
df_flights_test['std_op_carrier_delay'] = df_flights_test.apply(lambda row: df_std_op_carrier_delays[row['op_unique_carrier']] if row['op_unique_carrier'] in df_std_op_carrier_delays.index else df_std_op_carrier_delays.mean(), axis=1)

In [25]:
# Add relationship between tail number and delay time
tail_num_delay_means = df_flights.groupby('tail_num').mean()['arr_delay']
tail_num_delay_std = df_flights.groupby('tail_num').std()['arr_delay']

df_flights['mean_tail_num_arr_delay'] = df_flights.apply(lambda row: tail_num_delay_means[row['tail_num']] if row['tail_num'] in tail_num_delay_means.index else tail_num_delay_means.mean(), axis=1)
df_flights['std_tail_num_arr_delay'] = df_flights.apply(lambda row: tail_num_delay_std[row['tail_num']] if row['tail_num'] in tail_num_delay_std.index else tail_num_delay_std.mean(), axis=1)
df_flights['std_tail_num_arr_delay'].fillna(value=tail_num_delay_std.mean(),inplace=True)  # Fix null values

df_flights_test['mean_tail_num_arr_delay'] = df_flights_test.apply(lambda row: tail_num_delay_means[row['tail_num']] if row['tail_num'] in tail_num_delay_means.index else tail_num_delay_means.mean(), axis=1)
df_flights_test['std_tail_num_arr_delay'] = df_flights_test.apply(lambda row: tail_num_delay_std[row['tail_num']] if row['tail_num'] in tail_num_delay_std.index else tail_num_delay_std.mean(), axis=1)
df_flights_test['std_tail_num_arr_delay'].fillna(value=tail_num_delay_std.mean(),inplace=True)  # Fix null values

In [26]:
df_flights;

In [ ]:
# One hot encoding?

# Dropping Columns, Construct Final DFs

In [27]:
# Drop inappropriate columns (this should be the final step, after other feature engineering is complete).
# Except for the arr_delay column, df_flights_final should have the same columns as df_flights_test in order to use both as training/testing X values.

df_flights_final = df_flights.drop(columns=['carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'no_name', 'dep_time', 'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in', 
                                      'arr_time', 'cancelled', 'cancellation_code', 'diverted', 'actual_elapsed_time', 'air_time', 'first_dep_time', 'total_add_gtime', 'longest_add_gtime',
                                           'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_city_name', 'dest_city_name', 'dup', 'flights'])

# Orignal columns of df_flights_test:
# ['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num', 'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name', 'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance']

df_flights_test_final = df_flights_test.drop(columns=['branded_code_share', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_carrier_fl_num', 'origin_city_name', 'dest_city_name', 'dup', 'flights'])
df_flights_test_final = df_flights_test_final.dropna()   # Drop rows with missing tail numbers (235 rows, or 0.14%)

In [45]:
df_flights_final;

In [46]:
df_flights_test_final;

In [30]:
# Export DFs to csv
#df_flights_final.to_csv('df_flights_final.csv')
#df_flights_test_final.to_csv('df_flights_test_final.csv')

# Hypothesis

I think these will be the most important feature(s):
* mkt and op carrier delay stats (mean/std)
* daily and hourly delay stats
* weather delay stats

I think these will be the least important feature(s):
* tail number stats